In [ ]:
"""
Algorithm 1 LEO-Based EC Algorithm
1: Begin
2: // Initialization
3: g←0 ; // the generation index
4: Initialize population Xg and evaluate the fitness;
5: Initialize the weights of ANN randomly;
6: Initialize arch as an empty set; // to store solution pairs
7: While stop criteria not satisfied Do
8:		g←g+ 1;
9:		// Individual Evolution
10:		Sample r uniformly on [0,1];
11:		If g> 1 and r< lp Then
12:			newX ← Evolve Xg by learning-aided evolutionary operator;
13:		Else
14:			// use operators in traditional EC, e.g., PSO or DE
15:			newX ← Evolve Xg by traditional evolutionary operator;
16:		End If
17:		Evaluate the fitness of individuals in newX;
18:		// Selection
19:		Xg+1← selection among Xg and newX;
20:		// SEP Collection
21:		For each individual i in Xg+1 Do
22:			If Xg+1,i is better than Xg,i Then
23:				Add (Xg,i , Xg+1,i ) in arch;
24:			End If
25:		End For
26:		If number of SEPs in arch > arch_size Then
27:			arch ← the newest arch_size solution pairs;
28:		End If
29:		// Learning System Update
30:		Train the ANN with all data in arch for one epoch;
31:	End While
32:End
"""

In [1]:
import numpy as np
from pymoo.algorithms.moo.rvea import RVEA
from pymoo.problems import get_problem
from pymoo.core.population import Population
from pymoo.optimize import minimize
from keras.layers import Dense
from keras.models import Sequential
from pymoo.core.evaluator import Evaluator
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.operators.mutation.pm import PM
from pymoo.operators.crossover.sbx import SBX
import random

In [2]:
def normalize(data):
    min_val = -100
    max_val = 100
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data


def denormalize(data):
    min_val = -100
    max_val = 100
    denormalized_data = (data * (max_val - min_val)) + min_val
    return denormalized_data

In [3]:
def generate_model(n_var):
    model = Sequential()
    model.add(Dense((3*n_var), input_dim=n_var, activation='sigmoid'))
    model.add(Dense(n_var, activation='sigmoid'))

    model.compile(loss='mse',
                  optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
def dominates(p, q):
    return np.all(p <= q) and np.any(p < q)

In [5]:

def traditional_rvea(problem, ref_dirs, pop_size=1000, generation=1000, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    X = np.random.uniform(l_bound, h_bound, (pop_size, n_var))

    pop = Population.new("X", X)
    Evaluator().eval(problem, pop)

    algorithm = RVEA(ref_dirs, crossover=SBX(eta=15, prob=0.9),
                     mutation=PM(eta=20, prob=0.5), pop_size=pop_size, sampling=pop)
    res = minimize(problem, algorithm, ("n_gen", generation), verbose=verbose)

    return res.X, res.F

In [6]:
n_obj = 5
n_var = 100
l_bound = -100
h_bound = 100

problem = get_problem('dtlz3', n_var=n_var, n_obj=n_obj)
problem.xl = np.array([-100 for _ in range(100)])
problem.xu = np.array([100 for _ in range(100)])
ref_dirs = get_reference_directions("das-dennis", n_obj, n_partitions=12)

pop_traditional, pareto_traditional = traditional_rvea(
    problem, ref_dirs, pop_size=100, generation=100, n_var=100, l_bound=-100, h_bound=100, verbose=True)


Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        0 |    100 |             - |             -
     2 |      100 |     89 |  0.1513048195 |         ideal
     3 |      200 |    113 |  0.0427196543 |         nadir
     4 |      300 |    131 |  0.0446680419 |         ideal
     5 |      400 |    148 |  0.1111433836 |         nadir
     6 |      500 |    148 |  0.0177145047 |         ideal
     7 |      600 |    157 |  0.0283550824 |         nadir
     8 |      700 |    172 |  0.0282668874 |         nadir
     9 |      800 |    178 |  0.0245401110 |         ideal
    10 |      900 |    176 |  0.0406036048 |         ideal
    11 |     1000 |    162 |  0.0100914919 |         ideal
    12 |     1100 |    157 |  0.0277059820 |         ideal
    13 |     1200 |    176

In [7]:

def leo_based_rvea(problem, ref_dirs, pop_size=100, generation=100, lp=0.1, n_var=100, l_bound=-100, h_bound=100, verbose=True):

    pop = np.random.uniform(l_bound, h_bound, (pop_size, n_var))

    # pop = Population.new("X", X)
    fit = problem.evaluate(pop)
    # Evaluator().eval(problem, pop)

    model = generate_model(n_var)

    archive_x = []
    archive_y = []

    for g in range(generation):
        if g > 0 and random.random() < lp:
            pred = model.predict(normalize(pop), verbose=verbose)
            pop_new = denormalize(pred)
            fit_new = problem.evaluate(pop_new)
        else:
            algorithm = RVEA(ref_dirs, crossover=SBX(eta=15, prob=0.9),
                             mutation=PM(eta=20, prob=0.5), pop_size=pop_size, sampling=pop)
            res = minimize(problem, algorithm, ("n_gen", 1), verbose=False)
            pop_new = res.X
            fit_new = res.F
        for i in range(fit_new.shape[0]):
            if dominates(fit_new[i], fit[i]):
                archive_x.append(pop[i])
                archive_y.append(pop_new[i])
        pop = pop_new
        fit = fit_new
        if len(archive_x) > 0:
            archive_x = archive_x[-100:]
            archive_y = archive_y[-100:]
            model.fit(normalize(np.array(archive_x)), normalize(
                np.array(archive_y)), epochs=1, verbose=verbose)
        print(f"gen:  {g}, archive size: {len(archive_x)}", end='       \r')
    return pop, fit

In [8]:
n_obj = 5
n_var = 100
l_bound = -100
h_bound = 100

problem = get_problem('dtlz3', n_var=n_var, n_obj=n_obj)
problem.xl = np.array([-100 for _ in range(100)])
problem.xu = np.array([100 for _ in range(100)])

pop_leo, pareto_leo = leo_based_rvea(problem, ref_dirs, pop_size=100, generation=100, lp=0.1,
                                     n_var=n_var, l_bound=l_bound, h_bound=h_bound, verbose=False)

In [9]:
from pymoo.indicators.igd import IGD

# ref_dirs = UniformReferenceDirectionFactory(n_obj, n_partitions=10).do()
# ref_dirs = get_reference_directions("uniform", 5, n_partitions=12)

# get the Pareto front
pf = problem.pareto_front(ref_dirs)

igd = IGD(pf)

In [10]:
print("traditional: ", igd(pareto_traditional))
print("leo-based: ", igd(pareto_leo))

traditional:  32284403.377834808
leo-based:  248102531.2298478


In [3]:
from pymoo.problems.multi import dtlz
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.core.population import Population
from pymoo.optimize import minimize
from keras.layers import Dense
from keras.models import Sequential
from pymoo.operators.mutation.pm import PM as PolynomialMutation
from pymoo.operators.crossover.sbx import SBX as SimulatedBinaryCrossover
from pymoo.core.evaluator import Evaluator
import numpy as np
import random


class EvolutionarySolver:

    def init(self, num_objs, num_vars, lower_bound=-100, upper_bound=100):
        self.num_objs = num_objs
        self.num_vars = num_vars
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound

    def normalize_input(self, data):
        min_val = self.lower_bound
        max_val = self.upper_bound
        normalized_data = (data - min_val) / (max_val - min_val)
        return normalized_data

    def denormalize_input(self, data):
        min_val = self.lower_bound
        max_val = self.upper_bound
        denormalized_data = (data * (max_val - min_val)) + min_val
        return denormalized_data

    def build_neural_network(self):
        model = Sequential()
        model.add(Dense((3*self.num_vars),
                  input_dim=self.num_vars, activation='sigmoid'))
        model.add(Dense(self.num_vars))

        model.compile(loss='mse',
                      optimizer='adam', metrics=['accuracy'])
        return model

    def is_superior(self, p, q):
        return np.median(p) < np.median(q)

    def create_problem(self, problem_name):
        problem_class = getattr(dtlz, problem_name.capitalize())
        return problem_class(self.num_vars, self.num_objs)

    def evolve_on_problem(self, problem_name, ref_dirs, population_size=100, generations=100, learning_probability=0.1, verbose=True):
        problem = self.create_problem(problem_name)

        X = np.random.uniform(
            self.lower_bound, self.upper_bound, (population_size, self.num_vars))
        pop = Population.new("X", X)
        fit = problem.evaluate(X)
        Evaluator().eval(problem, pop)

        model = self.build_neural_network()

        archive_x = []
        archive_y = []

        for g in range(generations):
            if g > 0 and random.random() < learning_probability:
                pred = model.predict(
                    self.normalize_input(pop), verbose=verbose)
                pop_new = self.denormalize_input(pred)
                fit_new = problem.evaluate(pop_new)
            else:
                algorithm = MOEAD(
                    ref_dirs,
                    crossover=SimulatedBinaryCrossover(eta=15, prob=0.9),
                    mutation=PolynomialMutation(eta=20, prob=0.9),
                    sampling=pop,
                    n_neighbors=15,
                    prob_neighbor_mating=0.7,
                )
                res = minimize(problem, algorithm, ("n_gen", 1), verbose=False)
                pop_new = res.X
                fit_new = res.F

            for i in range(fit_new.shape[0]):
                if self.is_superior(fit_new[i], fit[i]):
                    archive_x.append(pop[i])
                    archive_y.append(pop_new[i])

            pop = pop_new
            fit = fit_new

            for i in range(len(pop)):
                if self.is_superior(fit_new[i], fit[i]):
                    archive_x.append(pop[i])
                    archive_y.append(pop_new[i])

            if len(archive_x) > population_size:
                archive_x = archive_x[-population_size:]
                archive_y = archive_y[-population_size:]

            if len(archive_x) > 0:
                model.fit(
                    self.normalize_input(np.array(archive_x)),
                    self.normalize_input(np.array(archive_y)),
                    epochs=1,
                    verbose=verbose
                )

            print(f"Generation: {g}, Archive Size: {len(archive_x)}\n")

        return pop, fit

ImportError: cannot import name 'dtlz' from 'pymoo.problems.multi' (/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pymoo/problems/multi/__init__.py)